In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [3]:
cd

/content


In [4]:
ls

datalab/


In [5]:
cd datalab

/content/datalab


In [6]:
ls

adc.json


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [9]:
cd

/content


In [10]:
ls

datalab/


In [11]:
cd datalab

/content/datalab


In [12]:
ls

adc.json  drive/


In [13]:
cd drive

/content/datalab/drive


In [14]:
ls

Colab Notebooks/                  training.ipynb
Getting started                   Untitled
imdb_db.mat                       Untitled0.ipynb
New Microsoft Word Document.docx  Untitled (8c7d0787)
Screenshot (549).odt              Untitled (c499e044)
Screenshot (549).png              weights.29-3.76_utk.hdf5


In [15]:
import h5py
import pandas as pd
import numpy as np
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from scipy.io import loadmat
from datetime import datetime
import os
from keras.models import load_model, Model
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
model = load_model("weights.29-3.76_utk.hdf5")

In [29]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 16)   432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 64, 64, 16)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
last_layer = model.get_layer('average_pooling2d_1').output

In [0]:
def load_data(path):
    d = loadmat(path)
    return d["image"], d["gender"][0], d["age"][0], d["db"][0], d["img_size"][0, 0], d["min_score"][0, 0]

In [20]:
print("Loading data...")
image, gender, age, _, image_size, _ = load_data("imdb_db")

Loading data...


In [0]:
X_data = image
y_data_g = np_utils.to_categorical(gender, 2)
y_data_a = np_utils.to_categorical(age, 101)

In [0]:
image_input = Input(shape=(64, 64, 3))
x= Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
predictions_g = Dense(2, activation='softmax', name='gender')(x)
predictions_a = Dense(101, activation='softmax', name='age')(x)

In [0]:
model_custom = Model(model.get_layer('input_1').output, [predictions_g, predictions_a])

In [35]:
model_custom.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 16)   432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 64, 64, 16)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
for layer in model_custom.layers[:-3]:
    layer.trainable = False

In [37]:
model_custom.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 16)   432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 64, 64, 16)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
model_custom.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train_g,y_test_g, y_train_a,y_test_a=train_test_split(X_data, y_data_g,y_data_a, test_size=0.15, random_state=42,shuffle=True)

In [46]:
len(X_train)

146074

In [47]:
len(y_train_g)

146074

In [48]:
len(y_train_a)

146074

In [49]:
len(y_test_g)

25778

In [50]:
len(y_test_a)

25778

In [51]:
model_custom.fit(X_train, [y_train_g, y_train_a],batch_size=512, epochs=20)

Epoch 1/20
146074/146074 [==============================] - 724s 5ms/step - loss: 3.7640 - gender_loss: 0.2360 - age_loss: 3.4374 - gender_acc: 0.9094 - age_acc: 0.0757
Epoch 2/20
146074/146074 [==============================] - 713s 5ms/step - loss: 3.6179 - gender_loss: 0.2266 - age_loss: 3.3008 - gender_acc: 0.9128 - age_acc: 0.0939
Epoch 3/20
146074/146074 [==============================] - 713s 5ms/step - loss: 3.5842 - gender_loss: 0.2240 - age_loss: 3.2696 - gender_acc: 0.9136 - age_acc: 0.1031
Epoch 4/20
146074/146074 [==============================] - 712s 5ms/step - loss: 3.5580 - gender_loss: 0.2217 - age_loss: 3.2457 - gender_acc: 0.9144 - age_acc: 0.1112
Epoch 5/20
146074/146074 [==============================] - 712s 5ms/step - loss: 3.5370 - gender_loss: 0.2200 - age_loss: 3.2264 - gender_acc: 0.9144 - age_acc: 0.1156
Epoch 6/20
146074/146074 [==============================] - 712s 5ms/step - loss: 3.5190 - gender_loss: 0.2183 - age_loss: 3.2101 - gender_acc: 0.9152 - ag

In [0]:
from keras.models import model_from_json

In [53]:
model_json = model_custom.to_json()
with open("model_json", "w") as json_file:
    json_file.write(model_json)
model_custom.save_weights("model_custom_weights.h5")
print("saved model to drive")

saved model to drive


In [54]:
ls

Colab Notebooks/                  Screenshot (549).png
Getting started                   training.ipynb
imdb_db.mat                       Untitled
model_custom_weights.h5           Untitled0.ipynb
model_json                        Untitled (8c7d0787)
New Microsoft Word Document.docx  Untitled (c499e044)
Screenshot (549).odt              weights.29-3.76_utk.hdf5
